# Spark DataFrame Missing Values

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("miss").getOrCreate()

In [4]:
df = spark.read.csv("Data/ContainsNull.csv", inferSchema=True, header=True)

In [5]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



## treating missing value 

In [13]:
df.na.drop().show() # we will delete any row that containt missing values
# but we can specify a thresh 
df.na.drop(thresh = 2 ).show() # it means that the row must have at least two missing values to be deleted 
# there another option by using How " any " delete any row that containt missing value " all " delete the row
# that contains only missing values 
df.na.drop(how = "any").show()


+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [16]:
# we Can consider only deleting according to some column 
df.na.drop(subset = ["Sales"]).show()  # delete any row missing in Sales

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### Fill the missing value 

In [18]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [37]:
df.na.fill("Fill Value").show()  # fill the string missing value in df 
df.na.fill(0).show()  # fill the int missing value in df 

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| null|
|emp2|Fill Value| null|
|emp3|Fill Value|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [38]:
df.na.fill("No name", subset=["Name"]).show() # its better to spicfy the columns 

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No name| null|
|emp3|No name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [39]:
from pyspark.sql.functions import mean

In [48]:
mean_val = df.select(mean(df["Sales"])).collect()


In [49]:
mean_val = mean_val[0][0]
mean_val

400.5

In [50]:
df.na.fill(mean_val, subset=["Sales"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [52]:
# in one line 
df.na.fill(df.select(mean(df["Sales"])).collect()[0][0],["Sales"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

